**LSTM with FastText**

In [ ]:
from tensorflow.keras.layers import BatchNormalization

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU,SimpleRNN
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from tensorflow.keras.layers import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from plotly import graph_objs as go
import plotly.express as ex
import plotly.figure_factory as ff

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
print('mount success')

In [ ]:
train = pd.read_csv('/content/drive/Shareddrives/EMOThreat/train_set_taskA.csv')
test = pd.read_csv('/content/drive/Shareddrives/EMOThreat/test_set_taskA.csv')
train.head()
test.head()

In [ ]:
print(train.shape)
print(test.shape)

cleaning

In [ ]:
import re,string

def strip_links(text):
    link_regex    = re.compile('((https?):((//)|(\\\\))+([\w\d:#@%/;$()~_?\+-=\\\.&](#!)?)*)', re.DOTALL)
    links         = re.findall(link_regex, text)
    for link in links:
        text = text.replace(link[0], ', ')    
    return text
train['Sentences']=train['Sentences'].apply(lambda x:strip_links(x))
test['Sentences']=test['Sentences'].apply(lambda x:strip_links(x))
### replace :\n 
train['Sentences']=train['Sentences'].str.replace("\n",' ')
### replace :\n 
test['Sentences']=test['Sentences'].str.replace("\n",' ')
# Define the function to remove the punctuation
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text
# Apply to the DF series
train['Sentences'] = train['Sentences'].apply(remove_punctuations) 
# Apply to the DF series
test['Sentences'] = test['Sentences'].apply(remove_punctuations) 

training

In [ ]:
list_classes = ["anger","disgust","fear","sadness","surprise","happiness","neutral"]
Y = train[list_classes].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train.Sentences.values, Y,  
                                                  random_state=42, 
                                                  test_size=0.2)

In [ ]:
train['Sentences'].apply(lambda x:len(str(x).split())).max()
max_features = 10000
maxlen = 500
token=tf.keras.preprocessing.text.Tokenizer(num_words=max_features)
token.fit_on_texts(train.Sentences)

In [ ]:
X_train_seq=token.texts_to_sequences(X_train)
X_test_seq=token.texts_to_sequences(X_test)

In [ ]:
#zero pad the sequences
X_train_pad = sequence.pad_sequences(X_train_seq, maxlen=maxlen)
X_test_pad = sequence.pad_sequences(X_test_seq, maxlen=maxlen)
word_index = token.word_index
len(token.word_index)

Fasttext Embeddings

In [ ]:
### remove unnecessary files
!rm -rf  ./cc.en.300.bin.gz
!rm -rf ./GoogleNews-vectors-negative300.bin.gz
!rm -rf ./glove.6B.300d.txt
!rm -rf ./glove.6B.200d.txt
!rm -rf ./glove.6B.100d.txt
!rm -rf ./glove.6B.zip

!git clone https://github.com/facebookresearch/fastText.git
!cd fastText
!pip install fastText
import fasttext.util
fasttext.util.download_model('ur', if_exists='ignore')  # Urdu

In [ ]:
### remove unnecessary files
!rm -rf  ./cc.en.300.bin.gz
!rm -rf ./GoogleNews-vectors-negative300.bin.gz
!rm -rf ./glove.6B.300d.txt
!rm -rf ./glove.6B.200d.txt
!rm -rf ./glove.6B.100d.txt
!rm -rf ./glove.6B.zip

In [ ]:
ft = fasttext.load_model('cc.ur.300.bin')

In [ ]:
fasttext.util.reduce_model(ft, 50)
embedding_vector_length=300
#Initialize embedding matrix
embedding_matrix_fasttext = np.zeros((max_features + 1, embedding_vector_length))
print(embedding_matrix_fasttext.shape)

In [ ]:
vocab_size = len(token.word_index) + 1
from keras.layers import Input
from keras.models import Model
deep_inputs = Input(shape=(maxlen,))
embedding_layer = Embedding(5001, 300, weights=[embedding_matrix_fasttext], trainable=False)(deep_inputs)
LSTM_Layer_1 = LSTM(128)(embedding_layer)
dense_layer_1 = Dense(7, activation='sigmoid')(LSTM_Layer_1)
model = Model(inputs=deep_inputs, outputs=dense_layer_1)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

In [ ]:
model.summary()

In [ ]:
for word, i in sorted(token.word_index.items(),key=lambda x:x[1]):
    if i > (max_features+1):
        break
    try:
        embedding_vector = ft[word] #Reading word's embedding from Glove model for a given word
        embedding_matrix_fasttext[i] = embedding_vector
    except:
        pass
# A simpleRNN without any pretrained embeddings and one dense layer
model = Sequential()
model.add(tf.keras.layers.Embedding(max_features + 1, #Vocablury size
                                    embedding_vector_length, #Embedding size
                                    weights=[embedding_matrix_fasttext], #Embeddings taken from pre-trained model
                                    trainable=False, #As embeddings are already available, we will not train this layer. It will act as lookup layer.
                                    input_length=maxlen) #Number of words in each review
         )
model.add(SimpleRNN(100))
model.add(Dense(7, activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

In [ ]:
history_fasttext=model.fit(X_train_pad,y_train,
                           epochs=5,
                           batch_size=32,          
                           validation_data=(X_test_pad, y_test))

In [ ]:
score = model.evaluate(X_test_pad, y_test, verbose=1)

print("Test Score:", score[0])
print("Test Accuracy:", score[1])

In [ ]:
import pandas as pd
import math
test['Sentences'].apply(lambda x:len(str(x).split())).max()
max_features = 10000
maxlen = 500
token=tf.keras.preprocessing.text.Tokenizer(num_words=max_features)
token.fit_on_texts(test.Sentences)

X = test["Sentences"]
X_test_seq=token.texts_to_sequences(X)
#zero pad the sequences
X_test_pad = sequence.pad_sequences(X_test_seq, maxlen=maxlen)
word_index = token.word_index
len(token.word_index)

results=model.predict(X_test_pad)
res = results.round()
res= res.astype(int)
csv = pd.DataFrame(res.tolist(),columns=["anger","disgust","fear","sadness","surprise","happiness","neutral"])
csv.insert(7,"Sentences",test["Sentences"],True)
csv.to_csv("lstm_w2v.csv", index=False)